# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
pip install -U scikit-learn

     |████████████████████████████████| 7.8 MB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 5.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install -U imbalanced-learn

  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)
     |████████████████████████████████| 8.6 MB 3.5 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U imbalanced-learn

     |████████████████████████████████| 199 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 199 kB 2.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [3]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [4]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
columns = ["home_ownership","verification_status","initial_list_status","application_type","issue_d","next_pymnt_d","pymnt_plan","hardship_flag","debt_settlement_flag"]
for column in columns:
           LE.fit(df[column])
           df[column] = LE.transform(df[column])
df.tail()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
68812,10000.0,0.1502,346.76,3,26000.0,1,1,low_risk,0,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,0,0
68813,12000.0,0.2727,368.37,3,63000.0,0,1,low_risk,0,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,0,0
68814,5000.0,0.1992,185.62,1,52000.0,1,1,low_risk,0,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,0,0
68815,40000.0,0.0646,1225.24,1,520000.0,2,1,low_risk,0,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,0,0
68816,16000.0,0.1131,350.36,1,72000.0,2,1,low_risk,0,7.02,...,94.3,0.0,1.0,0.0,251486.0,74835.0,23000.0,63090.0,0,0


In [5]:
# Create our features
X = df.drop(columns=["loan_status"])

# Create our target
y = df["loan_status"]


In [6]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [7]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(51612, 85)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
BRFC = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
BRFC.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [13]:
# Calculated the balanced accuracy score
Y_PRED_BRFC = BRFC.predict(X_test_scaled)
BAL_BRFC=balanced_accuracy_score(y_test, Y_PRED_BRFC)
print(BAL_BRFC)

0.7745204684723884


In [15]:
# Display the confusion matrix
CON_MAT_BRFC = confusion_matrix(y_test, Y_PRED_BRFC)
CM_DF_BRFC = pd.DataFrame(
    CON_MAT_BRFC, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
CM_DF_BRFC

,Predicted High Risk,Predicted Low Risk
Actual High Risk,56,31
Actual Low Risk,1620,15498


In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, Y_PRED_BRFC))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.64      0.91      0.06      0.76      0.57        87
   low_risk       1.00      0.91      0.64      0.95      0.76      0.60     17118

avg / total       0.99      0.90      0.65      0.94      0.76      0.60     17205



In [17]:
# List the features sorted in descending order by feature importance
importances = pd.DataFrame(BRFC.feature_importances_, index = X_train.columns, columns=['Importance']).sort_values('Importance', ascending=False)
importances

,Importance
total_rec_prncp,0.071913
total_rec_int,0.064014
total_pymnt_inv,0.060418
last_pymnt_amnt,0.060274
total_pymnt,0.055713
...,...
pymnt_plan,0.000000
acc_now_delinq,0.000000
delinq_amnt,0.000000
hardship_flag,0.000000


### Easy Ensemble Classifier

In [18]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=1000, random_state=1)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=1000, random_state=1)

In [19]:
# Calculated the balanced accuracy score
y_pred_eec = eec.predict(X_test_scaled)
bas_eec=balanced_accuracy_score(y_test, y_pred_eec)
print(bas_eec)

0.9257778664120446


In [20]:
# Display the confusion matrix
cm_eec = confusion_matrix(y_test, y_pred_eec)
cm_df_eec = pd.DataFrame(
    cm_eec, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_eec

,Predicted High Risk,Predicted Low Risk
Actual High Risk,79,8
Actual Low Risk,967,16151


In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [23]:
print(f'1.Easy Ensemble Classifier gave the best balance accuracy score of {bas_eec} compared to the one ran in the Balanced Random Forest Classifier of {BAL_BRFC}')

1.Easy Ensemble Classifier gave the best balance accuracy score of 0.9257778664120446 compared to the one ran in the Balanced Random Forest Classifier of 0.7745204684723884


In [24]:
print(f'2.Easy Ensemble Classifier gave the highest recall score with high_risk of 0.91, low_risk of 0.94, average/total of 0.94')

2.Easy Ensemble Classifier gave the highest recall score with high_risk of 0.91, low_risk of 0.94, average/total of 0.94


In [25]:
print(f'3.Easy Ensemble Classifier gives the higher geometric mean score of 0.93')

3.Easy Ensemble Classifier gives the higher geometric mean score of 0.93


In [26]:
print(f'Top 3 features are total_rec_prncp, total_rec_int and total_pymnt_inv')

Top 3 features are total_rec_prncp, total_rec_int and total_pymnt_inv
